# パターンマッチ

- Elixirの強力な構文
- Elixirにおける`=`演算子はマッチ演算子と呼ばれ、手続き型の言語の代入演算子とは概念が異なる

## マッチ演算子

- Elixirでは`=`はマッチ演算子であり、代入演算子ではない

In [ ]:
# 一番最初の時点ではxは定義されていないし、何の値にも束縛されていない
IO.puts x 
# => (CompileError) console:2 "undefined function x/0"

In [ ]:
x = 0
y = x
y = 1

# 以下では何が出力されるか, 予測して実行してみよう
[
  x != y,
  x,
  y
]
|> Enum.map(&IO.inspect/1)

In [ ]:
x = 0
0 = x # xは0で束縛されているのでマッチが成功

In [ ]:
x = 0
1 = x # => %MatchError{term: 0} xは既に0に束縛されているので、1とはマッチしない

### Listに対するパターンマッチ

In [ ]:
# 要素が全て等しいのでマッチが成功

l = [0, 1, 2, 3]
[0, 1, 2, 3] = l

In [ ]:
# 変数が含まれる場合、右辺にマッチするよう値が束縛される
[a, b, c, d] = [0, 1, 2, 3]

[a, b, c, d]
|> Enum.map(&IO.inspect/1)

In [ ]:
# 右辺が変数を含む場合、変数が束縛されている値にマッチする
x = 10
y = 20
[a, b, c, d] = [0, x, 2, y]

[a, b, c, d]
|> Enum.map(&IO.inspect/1)

Listに対するパターンマッチでは特有のマッチ記法がある。

`[a | b]` の `a` はリストの先頭、`b` は先頭以外の全てにマッチする。

In [ ]:
[head | tail] = [0, 1, 2, 3, 4]

[head, tail]
|> Enum.map(&IO.inspect/1)

In [ ]:
[head | tail] = [0]

[head, tail]
|> Enum.map(&IO.inspect/1)

In [ ]:
[head | tail] = []
# => %MatchError{term: []}

先頭から$n$個の要素に対してマッチすることも可能。

反対に、最後から$n$個のマッチは不可能。

In [ ]:
[first, second | tail] = [0, 1, 2, 3, 4]

[first, second, tail]
|> Enum.map(&IO.inspect/1)

In [ ]:
[head | last_one_before, last] = [0, 1, 2] # => CompileError

### Mapに対するパターンマッチ

- 左辺は右辺のサブセットであればいい
- ネストしたマップにもパターンマッチ可能

In [ ]:
%{x: x} = %{x: 0, y: 1, z: 2}
IO.inspect x

In [ ]:
%{:x => x} = %{x: 0, y: 1, z: 2}
IO.inspect x

In [ ]:
%{no_key: value} = %{x: 0, y: 1, z: 2}

In [ ]:
nested_map = %{
  outer_universe: %{
    universe: %{
      hello: "universe!",
      answer_of_everything: 42
    }
  }
}

%{outer_universe: %{universe: %{hello: target}}} = nested_map
IO.inspect target

# マッチ演算子をネストさせることもできる
%{
  outer_universe: %{
    universe: %{
      answer_of_everything: the_answer
    } = universe
  }
} = nested_map

IO.inspect universe
IO.inspect the_answer

In [ ]:
food_collection = %{
  red: %{
    meet: ["beef", "pork", "chicken", "mutton"]
  },
  green: %{
    vegitable: ["tomato", "carrot"],
    fruite: ["apple", "banana", "orange"]
  },
  yellow: %{
    fat: ["oil", "het", "rard"],
    carbohydrate: ["rice", "bread"]
  }
}

# try to extract and output meet list

# try to extract banana

# try to extract oil and yellow map at once

### pin演算子

- 変数は通常、別の値にマッチさせると新しい値に束縛される
- pin演算子を使うと、変数が束縛されている値に対してマッチするか試すことができる

In [ ]:
assistants = [
  %{name: "Alexa",  listen: "Hello"},
  %{name: "Siri",   listen: "Hey"},
  %{name: "Google", listen: "OK"},
]

your_message = "Hey"
Enum.each(assistants, fn %{name: name} = assistant -> 
  IO.puts "#{your_message}, #{name}!"
  case assistant do
    %{listen: your_message} -> IO.puts "#{name} responds to your #{your_message}"
    _                       -> IO.puts "Nothing happens..."
  end
end)


In [ ]:
assistants = [
  %{name: "Alexa",  listen: "Hello"},
  %{name: "Siri",   listen: "Hey"},
  %{name: "Google", listen: "OK"},
]

your_message = "Hey"
Enum.each(assistants, fn %{name: name} = assistant -> 
  IO.puts "#{your_message}, #{name}!"
  case assistant do
    %{listen: ^your_message} -> IO.puts "#{name} responds to your #{your_message}"
    _                        -> IO.puts "Nothing happens..."
  end
end)